# pivot_table melt

In [1]:
table = DataFrame({'cust_id':[10001,10001,10002,10002,10003],
                  'type':['normal','special_offer','normal','special_offer','special_offer'],
                  'monetary':[3608,420,1894,3503,4567]})
table

,cust_id,type,monetary
0,10001,normal,3608
1,10001,special_offer,420
2,10002,normal,1894
3,10002,special_offer,3503
4,10003,special_offer,4567


In [2]:
table1 = table.pivot_table(values='monetary',index='cust_id',columns='type',aggfunc='sum',fill_value=0)
table1.reset_index()

type,cust_id,normal,special_offer
0,10001,3608,420
1,10002,1894,3503
2,10003,0,4567


In [3]:
table2 = pd.melt(table1.reset_index(),id_vars='cust_id',value_vars=['normal','special_offer'],value_name='monetary',var_name='type')
table2

,cust_id,type,monetary
0,10001,normal,3608
1,10002,normal,1894
2,10003,normal,0
3,10001,special_offer,420
4,10002,special_offer,3503
5,10003,special_offer,4567


# cut

In [13]:
c = pd.cut(table2.monetary,bins=table2.monetary.quantile([0,0.5,1]),include_lowest=True)
c
table2.monetary.groupby(c).count()

monetary
(-0.001, 2698.5]    3
(2698.5, 4567.0]    3
Name: monetary, dtype: int64

In [14]:
cq = pd.qcut(table2.monetary,q=5)
cq
table2.monetary.groupby(cq).count()

0    (3503.0, 3608.0]
1     (420.0, 1894.0]
2     (-0.001, 420.0]
3     (-0.001, 420.0]
4    (1894.0, 3503.0]
5    (3608.0, 4567.0]
Name: monetary, dtype: category
Categories (5, interval[float64]): [(-0.001, 420.0] < (420.0, 1894.0] < (1894.0, 3503.0] < (3503.0, 3608.0] < (3608.0, 4567.0]]

monetary
(-0.001, 420.0]     2
(420.0, 1894.0]     1
(1894.0, 3503.0]    1
(3503.0, 3608.0]    1
(3608.0, 4567.0]    1
Name: monetary, dtype: int64

# RFM

In [27]:
def dateparse(x):
    return pd.datetime.strptime(x, '%d%b%y:%H:%M:%S')

data = pd.read_csv('./RFM_TRAD_FLOW.csv',encoding='gbk',parse_dates=['time'],date_parser=dateparse)
data[:5]

,transID,cumid,time,amount,type_label,type
0,9407,10001,2009-06-14 17:58:34,199.0,正常,Normal
1,9625,10001,2009-06-16 15:09:13,369.0,正常,Normal
2,11837,10001,2009-07-01 14:50:36,369.0,正常,Normal
3,26629,10001,2009-12-14 18:05:32,359.0,正常,Normal
4,30850,10001,2010-04-12 13:02:20,399.0,正常,Normal


In [37]:
RFM = data.groupby('cumid').agg({'time':'max','amount':['count','sum']}).reset_index()
RFM.columns = ['cumid','recency','frequency','monetary']
RFM[:3]

,cumid,recency,frequency,monetary
0,10001,2010-09-17 12:47:43,27,3334.0
1,10002,2010-07-03 11:43:16,18,1652.0
2,10003,2010-08-28 16:04:25,25,3435.0


In [46]:
M_pivot = pd.pivot_table(data,index='cumid',columns='type',values='amount',aggfunc='sum',fill_value=0).reset_index()
M_pivot['spe_ratio'] = M_pivot.Special_offer/(M_pivot.Special_offer + M_pivot.Normal)
M_pivot.sort_values(by='spe_ratio',ascending=False)[:5]

type,cumid,Normal,Presented,Special_offer,returned_goods,spe_ratio
150,10151,765.0,0,870.0,0.0,0.532110
932,40033,1206.0,0,761.0,-848.0,0.386884
1135,40236,1155.0,0,691.0,-793.0,0.374323
824,30225,1475.0,0,738.0,-301.0,0.333484
367,20068,1631.0,0,731.0,-239.0,0.309483
